In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset

from torch import float32


# ✅ Charger le dataset JSONL
dataset = load_dataset('json', data_files='data/dataset.jsonl')

# ✅ Vérifier les clés du dataset
print("Exemple de données :", dataset['train'][0])  # Affiche un exemple pour vérifier sa structure


c:\Users\Quera\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Exemple de données : {'messages': [{'role': 'system', 'content': 'Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.'}, {'role': 'user', 'content': 'Quelles sont vos expertises ?'}, {'role': 'assistant', 'content': "Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs."}]}


In [4]:
dataset['train'][0]

{'messages': [{'role': 'system',
   'content': 'Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.'},
  {'role': 'user', 'content': 'Quelles sont vos expertises ?'},
  {'role': 'assistant',
   'content': "Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs."}]}

In [5]:
# ✅ Fonction de prétraitement
def format_conversation(example):
    return {
        "text": "\n".join(f"{'Utilisateur' if msg['role'] == 'user' else 'Quera'}: {msg['content']}" 
                           for msg in example["messages"]).strip()
    }


# ✅ Appliquer la transformation
dataset = dataset.map(format_conversation)

Map: 100%|██████████| 10/10 [00:00<00:00, 240.50 examples/s]


In [6]:
dataset['train'][0]

{'messages': [{'role': 'system',
   'content': 'Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.'},
  {'role': 'user', 'content': 'Quelles sont vos expertises ?'},
  {'role': 'assistant',
   'content': "Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs."}],
 'text': "Quera: Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.\nUtilisateur: Quelles sont vos expertises ?\nQuera: Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs."}

In [8]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA n'a pas de token de padding

In [9]:
tokenizer.eos_token

'<|eot_id|>'

In [7]:
# ✅ Fonction de prétraitement avec labels
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=40)
    
    # ✅ Copier input_ids pour labels
    inputs["labels"] = inputs["input_ids"].copy()

    # ✅ Remplacer les tokens de padding par -100 (Hugging Face ignore ces positions dans la perte)
    padding_token_id = tokenizer.pad_token_id
    inputs["labels"] = [
        [(label if label != padding_token_id else -100) for label in labels] for labels in inputs["labels"]
    ]

    return inputs

In [10]:
# ✅ Appliquer la transformation
dataset = dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x000001E583D86B60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 10/10 [00:00<00:00, 218.24 examples/s]


In [18]:
dataset['train'][0]

{'messages': [{'role': 'system',
   'content': 'Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.'},
  {'role': 'user', 'content': 'Quelles sont vos expertises ?'},
  {'role': 'assistant',
   'content': "Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs."}],
 'text': "Quera: Vous êtes Quera, une agence spécialisée en Web et Data, située à Paris.\nUtilisateur: Quelles sont vos expertises ?\nQuera: Nous sommes experts en création de sites WordPress, SEO, Data Science, IA et développement d'APIs.",
 'input_ids': [128000,
  2232,
  2473,
  25,
  41621,
  62299,
  3489,
  2473,
  11,
  6316,
  945,
  768,
  76305,
  285,
  8047,
  665,
  5000,
  1880,
  2956,
  11,
  10109,
  8047,
  3869,
  12366,
  627,
  2810,
  33226,
  25,
  3489,
  37907,
  15132,
  26317,
  6335,
  5014,
  18072,
  2232,
  2473,
  25,
  48109,
  74533],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1